In [ ]:
import pickle

import lightgbm
from lightgbm import LGBMRegressor
from sklearn.linear_model import LogisticRegression

from lightgbm import LGBMClassifier
import csv
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import KFold, RandomizedSearchCV, GridSearchCV
import xgboost as xgb
# data normalization with sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
import missingno as msno


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/fifa-20-complete-player-dataset/players_20.csv', index_col = False)


In [ ]:
useless_col = ['dob','sofifa_id','player_url','long_name','body_type','real_face','nation_position','loaned_from','nation_jersey_number']
df.drop(useless_col, inplace=True, axis=1)


In [ ]:
pos_nl = df[df['player_positions']=='GK'].index.tolist()
df.drop(df.index[pos_nl],inplace= True)

In [ ]:
gk_cols = ['goalkeeping_diving',
       'goalkeeping_handling', 'goalkeeping_kicking',
       'goalkeeping_positioning', 'goalkeeping_reflexes', 'gk_diving', 'gk_handling', 'gk_kicking', 'gk_reflexes', 'gk_speed',
       'gk_positioning']
df.drop(gk_cols, inplace=True, axis=1)


In [ ]:
pos_cols = ['ls','st','rs','lw','lf','cf','rf','rw','lam','cam','ram','lm','lcm','cm','rcm','rm','lwb','ldm', 'cdm','rdm','rwb','lb','lcb','cb','rcb','rb']
for col in pos_cols:
  df[col]=df[col].str.split('+',n=1,expand = True)[0]



In [ ]:
df

In [ ]:
msno.bar(df,(20,5),color='green', labels=True, fontsize= 10)


In [ ]:
for col in df.columns:
    if df[col].isnull().sum()!=0:
        print (col,df[col].isnull().sum() )

In [ ]:
null_cols = ['release_clause_eur', 'player_tags','team_position','team_jersey_number', 'joined', 'contract_valid_until','player_traits']
df.drop(null_cols, inplace=True, axis=1)


In [ ]:
df[pos_cols] = df[pos_cols].astype(int)


In [ ]:
ex_cols = ['player_positions', 'short_name','nationality', 'club', 'preferred_foot']
df.drop(ex_cols, inplace=True, axis=1)


In [ ]:
df[['Attack Work Rate', 'Defense Work Rate']] = df['work_rate'].str.split('/', 1, expand=True)
df.drop(['work_rate'],axis=1,inplace=True)
set(df['Defense Work Rate'])

In [ ]:
cat_columns = ['Attack Work Rate', 'Defense Work Rate' ]
for col in cat_columns:
  df[col] = df[col].astype('int')
  df[col] = df[col]+1
#df[cat_columns] = df[cat_columns].apply(lambda x: x.cat.codes)

In [ ]:
for col in df.select_dtypes(include='float').keys().tolist():
  print (col, ': ', len(set(df[col])))
  if df[col].apply(float.is_integer).all() == True:
    df[col] = df[col].astype(int)

In [ ]:
df.info()

In [ ]:
df.to_csv('player_20_clean.csv', index = False)